In [1]:
#baseline版本改进版第一版 利用osv特征
import  pandas as pd
train = pd.read_csv('./train.csv')
test1 = pd.read_csv('./test1.csv')

#去掉不参与建模的列
features = train.drop(['Unnamed: 0','label'],axis=1)#去掉id和label
labels = train['label']
features.columns

#先辨别出来类别特征
cate_feture = ['apptype','carrier','ntt','version','location','cus_type']

#不参与建模的特征[os,lan,sid]
remove_list = ['os','lan','sid']
col = features.columns.tolist()
for i in remove_list:
    col.remove(i)

#特征筛选
features = features[col]

#开始对timediff进行优化
#先将原字段/1000 再做转换日期格式
import time
from datetime import datetime
def get_date(features):
    features['timestamp'] = features['timestamp'].apply(lambda x:datetime.fromtimestamp(x/1000))
    # 创建时间戳索引
    temp = pd.DatetimeIndex(features['timestamp'])
    features['day'] = temp.day
    features['hour'] = temp.hour

    #添加timediff
    start_time = features['timestamp'].min()
    features['timediff'] = features['timestamp'] - start_time
    #将timediff转化为小时类型
    features['timediff'] = features['timediff'].dt.days * 24 + features['timediff'].dt.seconds/3600
    features.drop(['timestamp'],axis=1,inplace=True)
    return features


#对训练集进行时间提取
features = get_date(features)
#对测试集进行时间提取
test1 = get_date(test1)

#对osv进行labelencoder特征编码
#需要将测试集和训练集合并做labelencoder 因为可能会出现再训练集中有但是测试及中没有出现过的情况
all_df = pd.concat([train,test1])
all_df['osv'] = all_df['osv'].astype('str')
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
all_df['osv'] = le.fit_transform(all_df['osv'])

#将训练完的osv进行还原
all_df[all_df['label'].notnull()]['osv']

#特征工程
#对于数值类型特别大的异常值设置为0  只有小于16位的为值本身，超过16为的则设置为0
features['fea_hash'] = features['fea_hash'].map(lambda x:0 if len(str(x)) > 16 else int(x))
features['fea1_hash'] = features['fea1_hash'].map(lambda x:0 if len(str(x)) > 16 else int(x))
features['fea_hash'].value_counts()
#对于version列来说，有一些非数值类型的值，设置为0，（只是暂定的一个规划，不用纠结为啥）
features['version'] = features['version'].map(lambda x: int(x) if str(x).isdigit() else 0)
features['version'].value_counts()
features['osv'] = all_df[all_df['label'].notnull()]['osv']

#测试集预测 保持与features中的columns一致即可
test_fea = test1[features.columns]
#对于数值类型特别大的异常值设置为0  只有小于16位的为值本身，超过16为的则设置为0
test_fea['fea_hash'] = test_fea['fea_hash'].map(lambda x:0 if len(str(x)) > 16 else int(x))
test_fea['fea1_hash'] = test_fea['fea1_hash'].map(lambda x:0 if len(str(x)) > 16 else int(x))
#对于version列来说，有一些非数值类型的值，设置为0，（只是暂定的一个规划，不用纠结为啥）
test_fea['version'] = test_fea['version'].map(lambda x: int(x) if str(x).isdigit() else 0)
test_fea['osv'] = all_df[all_df['label'].isnull()]['osv']

from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.metrics import accuracy_score
def esamble_model(clf,train_x,train_y,test):
    prob=[] #记录最终结果
    mean_acc = 0 #记录平均准确率
    #采用5折交叉验证
    sk = StratifiedKFold(n_splits=5,random_state=2021,shuffle=True)
    for k,(train_index,val_index) in enumerate(sk.split(train_x,train_y)):
        train_x_real = train_x.iloc[train_index]
        train_y_real = train_y.iloc[train_index]
        val_x = train_x.iloc[val_index]
        val_y = train_y.iloc[val_index]
        #模型训练
        clf = clf.fit(train_x_real,train_y_real)
        val_y_pred = clf.predict(val_x)
        #子模型评估
        acc_val = accuracy_score(val_y,val_y_pred)
        print('第{}个子模型 acc{}'.format(k+1,acc_val))
        mean_acc += acc_val/5
        #子模型预测
        test_y_pred = clf.predict_proba(test)[:,-1] #取概率值
        prob.append(test_y_pred)
    print(mean_acc)
    mean_prob = sum(prob)/5
    return mean_prob

#使用XGBoost训练
import xgboost as xgb
clf = xgb.XGBClassifier(
            max_depth=6, learning_rate=0.05, n_estimators=2000, 
            objective='binary:logistic', tree_method='gpu_hist', 
            subsample=0.8, colsample_bytree=0.8, 
            min_child_samples=3, eval_metric='auc', reg_lambda=0.5
        )

result = esamble_model(clf,features,labels,test_fea)

#预测
a = pd.DataFrame(test1['sid'])
a['label']= result
a['label'] = a['label'].apply(lambda x:0 if x<0.5 else 1)
a.to_csv('versionfinal_xgb_final.csv',index=False)





D:\python\lib\site-packages\ipykernel_launcher.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\python\lib\site-packages\ipykernel_launcher.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\python\lib\site-packages\ipykernel_launcher.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retur

[21:02:49] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { min_child_samples } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


第1个子模型 acc0.88762
[21:03:35] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { min_child_samples } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


第2个子模型 acc0.88851
[21:04:18] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { min_child_samples } might not be used.

  This may not be 